In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import os
import re
import scipy as sci


In [2]:
def list_o_list(big_list, lil_size):
    l_o_l = []
    for i in range(0, len(big_list), lil_size):
        lil = big_list[i : min(i + lil_size, len(big_list))]
        l_o_l.append(lil)
    return(l_o_l)

In [3]:
test_list  = list(range(1,33,4))
test_chop = 9
print(list_o_list(test_list, test_chop))


[[1, 5, 9, 13, 17, 21, 25, 29]]


In [4]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 '01-08-19-V118',
 '03-05-19-V118',
 '03-26-19-V118',
 '08-14-18-V118',
 'EDA.ipynb',
 'GPS Files',
 'July 6, 2020 Vehicle 118 Expo Line Test Train Data.xlsx',
 'Skytrain Linear Asset Database - Consolidated.xlsx',
 'SMC DATALOG FOR MAR. 5 TEST TRAIN 118-6.xlsx',
 'Switch Maint Works to date - April 2019.xlsx',
 'Test Train Sample Noise Data',
 'Test Train Sample Noise Data.zip']

In [5]:
home = os.getcwd()


In [6]:
os.chdir("03-05-19-V118")
os.getcwd()

'C:\\Users\\tnank\\Documents\\GitHub\\project-sturm_drang\\03-05-19-V118'

In [7]:
files = os.listdir()
csv_files = []
for n in files:
    if re.search(".csv", n):
        csv_files.append(n)

In [ ]:
df = pd.read_csv(csv_files[0], header=15, names=["time", "acceleration_a"])
df.head()
df2 = pd.read_csv(csv_files[1], header=15, names=["time", "acceleration_b"])
df2.head()
raw = pd.merge(left=df, right=df2, how = "outer")

In [ ]:
del df
del df2

find the relationship between velocity and amplitude
then use the sections of higher quality gps signal to establish the realtionship between velocity and amplitude.

try on 20-30 second fourier transform.

try using

power spectrum/frequency analysis of each accelerometer.

clustering (historical and spacial) expect 5(ish) clusters, stoped, linear tavel, cornering, acceleration/decceleration. (curve radius)  engine/motor (also wheel constant) effects. 

silhouete

break all data files down to one particular consistent section of track

decision tree model for velocity (gps) prediction

first intergral of low frequency acceleration may give velocity.



In [ ]:

sb.scatterplot(x="time", y="acceleration_a", data= raw.loc[raw.time<600,:])

In [ ]:
raw.info()

In [ ]:
raw.describe()

In [ ]:
plt.figure(figsize=(25,10))
sb.scatterplot(x="time", y="acceleration_a", data = raw.loc[raw.time<300,:], alpha=0.2)
sb.scatterplot(x="time", y="acceleration_b", data = raw.loc[raw.time<300,:], alpha=0.2)
plt.show()

take first 25 mins for saftey then identify terminus 
abreviate to stoped in yard to stopped in terminus


In [ ]:
safe_len =60*25
dev = raw.loc[raw.time<safe_len,:]

In [ ]:
plt.figure(figsize=(25,10))
sb.scatterplot(x="time", y="acceleration_a", data = dev, alpha=0.2)
sb.scatterplot(x="time", y="acceleration_b", data = dev, alpha=0.2)
plt.show()

In [ ]:
safe_len

In [ ]:
safe_len/60


In [ ]:
dev.info()

In [ ]:
dev["abs_a"] = dev["acceleration_a"].abs()
dev["abs_b"] = dev.acceleration_b.abs()
dev.describe()

In [ ]:
cols = list(dev.columns)
cols.remove("time")
plt.figure(figsize=(25,10))
for n in cols:
    sb.scatterplot(x="time", y=n, data=dev.loc[dev.time <1800,:], alpha=0.2)
plt.show()

In [ ]:
y_a = np.fft.fft(dev.acceleration_a)

In [ ]:
type(y_a)

In [ ]:
len(y_a)

In [ ]:
len(dev)

In [ ]:
plt.scatter(np.arange(0 , len(y_a)), y_a)
plt.show()
#chunk = raw.loc[(raw.time>3600)&(raw.time<4200),:]
chunk = raw.copy()

In [ ]:
chunk.info()

In [ ]:
p2 = abs(y_a/len(dev))


In [ ]:
# Python example - Fourier transform using numpy.fft method

import numpy as np

import matplotlib.pyplot as plotter

 

# How many time points are needed i,e., Sampling Frequency

samplingFrequency   = 100;

 

# At what intervals time points are sampled

samplingInterval       = 1 / samplingFrequency;

 

# Begin time period of the signals

beginTime           = 0;

 

# End time period of the signals

endTime             = 10; 

 

# Frequency of the signals

signal1Frequency     = 4;

signal2Frequency     = 7;

 

# Time points

time        = np.arange(beginTime, endTime, samplingInterval);

 

# Create two sine waves

amplitude1 = np.sin(2*np.pi*signal1Frequency*time)

amplitude2 = np.sin(2*np.pi*signal2Frequency*time)

 

# Create subplot

figure, axis = plotter.subplots(4, 1)

plotter.subplots_adjust(hspace=1)

 

# Time domain representation for sine wave 1

axis[0].set_title('Sine wave with a frequency of 4 Hz')

axis[0].plot(time, amplitude1)

axis[0].set_xlabel('Time')

axis[0].set_ylabel('Amplitude')

 

 

# Time domain representation for sine wave 2

axis[1].set_title('Sine wave with a frequency of 7 Hz')

axis[1].plot(time, amplitude2)

axis[1].set_xlabel('Time')

axis[1].set_ylabel('Amplitude')

 

# Add the sine waves

amplitude = amplitude1 + amplitude2

 

# Time domain representation of the resultant sine wave

axis[2].set_title('Sine wave with multiple frequencies')

axis[2].plot(time, amplitude)

axis[2].set_xlabel('Time')

axis[2].set_ylabel('Amplitude')

 

# Frequency domain representation

fourierTransform = np.fft.fft(amplitude)/len(amplitude)           # Normalize amplitude

fourierTransform = fourierTransform[range(int(len(amplitude)/2))] # Exclude sampling frequency

 

tpCount     = len(amplitude)

values      = np.arange(int(tpCount/2))

timePeriod  = tpCount/samplingFrequency

frequencies = values/timePeriod

 

# Frequency domain representation

axis[3].set_title('Fourier transform depicting the frequency components')

 

axis[3].plot(frequencies, abs(fourierTransform))

axis[3].set_xlabel('Frequency')

axis[3].set_ylabel('Amplitude')

 

plotter.show()

In [ ]:
# Python example - Fourier transform using numpy.fft method

import numpy as np

import matplotlib.pyplot as plotter

 

# How many time points are needed i,e., Sampling Frequency

samplingFrequency   = 5120;

 

# At what intervals time points are sampled

samplingInterval       = 1 / samplingFrequency;

 

# Begin time period of the signals

beginTime           = 1830;

 

# End time period of the signals

endTime             = 1835; 

 

# Frequency of the signals

signal1Frequency     = 4;

signal2Frequency     = 7;

 

# Time points

time        = np.arange(beginTime, endTime, samplingInterval);
print(len(time))
print(1800*5120)

 

# Create two sine waves

amplitude1 = np.sin(2*np.pi*signal1Frequency*time)

amplitude2 = np.sin(2*np.pi*signal2Frequency*time)

 

# Create subplot



 



 

# Add the sine waves

amplitude = chunk.loc[(chunk.time<endTime)&(chunk.time>beginTime),"acceleration_a"]

 

# Time domain representation of the resultant sine wave


 

# Frequency domain representation

fourierTransform = np.fft.fft(amplitude)/len(amplitude)           # Normalize amplitude

fourierTransform = fourierTransform[range(int(len(amplitude)/2))] # Exclude sampling frequency

 

tpCount     = len(amplitude)

values      = np.arange(int(tpCount/2))

timePeriod  = tpCount/samplingFrequency

frequencies = values/timePeriod

 

# Frequency domain representation


plt.figure(figsize=(15,10))
sb.scatterplot(frequencies, abs(fourierTransform), alpha =0.2)

plt.show()
plt.figure(figsize=(15,10))
sb.scatterplot(x="time", y="acceleration_a", data=chunk.loc[(chunk.time<endTime)&(chunk.time>beginTime),:], alpha = 0.2)
plt.show()

feq = pd.DataFrame(data = {"freq":frequencies})

In [ ]:
a,b = freq_analysis(sample = chunk.loc[(chunk.time<endTime)&(chunk.time>beginTime),"acceleration_a"], time = endTime - beginTime, sampfrq = 5120, responsefrq= 1)

sb.scatterplot(x=a, y=b, alpha=0.2)

In [ ]:
feq.info()
feq["amp"] = abs(fourierTransform)

In [ ]:
print(feq.loc[feq.amp>0.0005,:])
freqs = np.arange(0, 1 + int(feq.freq.max()), 1)
amps = []
for n in freqs:
    cu = feq.loc[(feq.freq > n ) & (feq.freq <= n + 1),"amp"].sum()
    amps.append(cu)
sum_feq = pd.DataFrame( data = {"frequency":freqs, "amplitude":amps})

plt.figure(figsize=(15,10))
sb.scatterplot(x= "frequency", y = "amplitude", data=sum_feq, alpha = 0.2)
plt.show()
print(sum_feq.loc[sum_feq.amplitude>0.01,:])
sum_feq.head()


In [ ]:
a,b = freq_analysis(sample = chunk.loc[(chunk.time<endTime)&(chunk.time>beginTime),"acceleration_a"], time = endTime - beginTime, sampfrq = 5120, responsefrq= 10)

print(len(a), len(b))


In [ ]:
b[0:15]

In [ ]:
def freq_analysis(sample , time, responsefrq =1, sampfrq=5120):
    frqs = ""
    amps = []
    samp_int = 1/sampfrq
    ft = np.fft.fft(sample)/len(sample)
    ft = ft[range(int(len(sample)/2))]
    tpCount = len(sample)
    values = np.arange(int(tpCount/2))
    timePeriod = tpCount/samplingFrequency
    frequencies = values/timePeriod
    ft = abs(ft)
    print(len(ft))
    print(len(frequencies))
    feq = pd.DataFrame(data = {"freq":frequencies})
    
    feq["amp"] = ft
    
    frqs = np.arange(0, 1 + int(feq.freq.max()), responsefrq)
    print(len(frqs))
    for n in freqs:
        cu = feq.loc[(feq.freq > n ) & (feq.freq <= n + responsefrq),"amp"].sum()
        if len(cu) > 1:
            print("borked")
        amps.append(cu)
    
    return(frqs, amps)

In [ ]:
plt.figure(figsize=(20,20))
sb.scatterplot(x = "time", y = "acceleration_a", data = chunk, alpha= 0.2)
plt.show()

In [ ]:
type(fourierTransform)
len(fourierTransform)

In [ ]:
plt.figure(figsize=(15,10))
sb.scatterplot(frequencies, abs(fourierTransform), alpha =0.2)

plt.show()
plt.figure(figsize=(15,10))
sb.scatterplot(x = "time", y = "acceleration_a", data=chunk.loc[(chunk.time<endTime)&(chunk.time>beginTime),:], alpha = 0.2)
plt.show()

In [ ]:
print(dev.loc[(dev.time >0.995) & (dev.time < 1.005),:])

In [ ]:
chunk.info()

In [ ]:
range(int(len(amplitude)/2))

In [ ]:
chunk.plot(y="time")
plt.show()
print(chunk.info())

In [ ]:
raw.iloc[9215995:9216005,:]

In [ ]:
chunk.head()

In [ ]:
chunk.tail()
chunk= raw.copy()

In [ ]:
chunk["mins"] = chunk.time // 60

In [ ]:
chunk["hours"] = chunk.time // 3600

In [ ]:
chunk.head()

In [ ]:
chunk.tail()

In [ ]:
chunk.info()

In [ ]:
chunk.describe()

In [ ]:
chunk.info()